In [ ]:
from copy import copy 
from matplotlib import pyplot as plt
import datetime

import summer2
from summer.utils import ref_times_to_dti

from autumn.core.project import get_project, load_timeseries
from autumn.core.project.params import get_with_nested_key

# Targets represent data we are trying to fit to
from estival import targets as est
# We specify parameters using (Bayesian) priors
from estival import priors as esp
# Finally we combine these with our summer2 model in a BayesianCompartmentalModel (BCM)
from estival.model import BayesianCompartmentalModel

# Some preliminary code

### Load project, model and targets

In [ ]:
project = get_project("sm_covid2", "france")
death_target = project.calibration.targets[0]

default_params = project.param_set.baseline
m = project.build_model(default_params.to_dict()) 

In [ ]:
death_target.data =  death_target.data[death_target.data > 100].loc[:700]

In [ ]:
targets = [
    # est.NegativeBinomialTarget(name="infection_deaths", data=death_target.data, dispersion_param=100.)
    est.NormalTarget(name="infection_deaths", data=death_target.data, stdev=100.)
]

### Format priors to use in estival

In [ ]:
def get_estival_uniform_priors(autumn_priors):
    estival_priors = []
    for prior_dict in autumn_priors:
        assert prior_dict["distribution"] == "uniform", "Only uniform priors are currently supported"
        estival_priors.append(
            esp.UniformPrior(prior_dict["param_name"], prior_dict["distri_params"]),
        ) 
    return estival_priors
    

In [ ]:
priors = get_estival_uniform_priors(project.calibration.all_priors)

### Create BayesianCompartmentalModel object

In [ ]:
bcm = BayesianCompartmentalModel(m, default_params.to_dict(), priors, targets)

### Helper function to plot model fit to data

In [ ]:
def plot_fit(bcm, params):

    REF_DATE = datetime.date(2019,12,31)
    datetime_target = copy(bcm.targets["infection_deaths"].data)
    datetime_target.index = ref_times_to_dti(REF_DATE, datetime_target.index)

    ax = bcm.run(params).derived_outputs["infection_deaths"].plot()
    datetime_target.plot(style='.')
    ll = bcm.loglikelihood(**params)

    text = f"ll={round(ll, 4)}"
    plt.text(0.8, 0.9, text, transform=ax.transAxes)

# Compare model fits

In [ ]:
mle_file_params = {
    "infection_deaths_dispersion_param": 62.54504840986004,
    "infectious_seed_time": 30.447925441292778,
    "random_process.delta_values(1)": -1.306496185258189,
    "random_process.delta_values(10)": 0.12138039221640762,
    "random_process.delta_values(11)": 0.6411075116621299,
    "random_process.delta_values(2)": 0.11250765346332026,
    "random_process.delta_values(3)": 0.5009919549420943,
    "random_process.delta_values(4)": -0.06375110616249913,
    "random_process.delta_values(5)": -0.24028730546991617,
    "random_process.delta_values(6)": 0.2606697551298094,
    "random_process.delta_values(7)": -0.13194034829544132,
    "random_process.delta_values(8)": -0.3559750720412327,
    "random_process.delta_values(9)": 0.04754597441674813,
    "random_process.noise_sd": 0.5835215881701278,
    "voc_emergence.delta.new_voc_seed.time_from_gisaid_report": 18.52679245171894
}

In [ ]:
suggested_params = {}
for param_string in bcm.priors:
    try:
        suggested_params[param_string] = get_with_nested_key(default_params.to_dict(), param_string)
    except:
        if param_string in mle_file_params:
            suggested_params[param_string] = mle_file_params[param_string]
        else:
            print(f"No default value found for {param_string}")


In [ ]:
plot_fit(bcm, suggested_params)

In [ ]:

flat_epi_params = copy(suggested_params)
flat_epi_params.update({"contact_rate": 0.0})

In [ ]:
plot_fit(bcm, flat_epi_params)